# Lesson 3 Hyperparameter Tuning Using GridSearchCV

Welcome to today's lesson on **Hyperparameter Tuning Using GridSearchCV**! Our goal is to optimize a Gradient Boosting model to predict Tesla ($TSLA) stock prices more accurately. This lesson will guide you through the process of hyperparameter tuning using GridSearchCV, focusing on understanding key hyperparameters, setting up a hyperparameter grid, and implementing GridSearchCV to find better model parameters.

---

## Brief Revision of Loading and Preparing the Dataset

Before diving into hyperparameter tuning, let's quickly revise how we load and prepare our dataset. We start by loading the Tesla dataset, adding technical indicators, and splitting the data into training and testing sets.

Here's a quick overview of the code:

```python
import pandas as pd
from datasets import load_dataset

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Feature Engineering
tesla_df['SMA_5'] = tesla_df['Adj Close'].rolling(window=5).mean()
tesla_df['SMA_10'] = tesla_df['Adj Close'].rolling(window=10).mean()
tesla_df['EMA_5'] = tesla_df['Adj Close'].ewm(span=5, adjust=False).mean()
tesla_df['EMA_10'] = tesla_df['Adj Close'].ewm(span=10, adjust=False).mean()

# Drop NaN values created by moving averages
tesla_df.dropna(inplace=True)

# Select features and target
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_10', 'EMA_5', 'EMA_10']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values  # Predicting next day's close price
features = features[:-1]  # Align features and target arrays correctly for time series forecasting

# Splitting the dataset into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)
```

The code above loads the Tesla historic prices dataset, applies feature engineering to add technical indicators like Simple Moving Averages (SMA) and Exponential Moving Averages (EMA), and preprocesses the dataset by removing NaN values. It then selects relevant features and the target variables, preparing the data for training and testing by splitting it into training and testing sets. The line `target = tesla_df['Adj Close'].shift(-1).dropna().values` is used for predicting the next day's closing price. The line `features = features[:-1]` ensures that the features and target arrays are aligned correctly for a time series forecasting task where you want to predict the next day's closing price.

---

## Introduction to Hyperparameter Tuning

Hyperparameters are configuration settings used to tune how our models learn. Examples include `learning_rate`, `n_estimators`, and `max_depth` in Gradient Boosting. Proper hyperparameter tuning can significantly improve model performance.

Imagine you're trying to make the perfect soup. Hyperparameter tuning is like adjusting the seasoning to get the best flavor. Just like too much salt or too little pepper can ruin the dish, poor hyperparameters can underperform our model.

The downside of this approach, however, is that it takes much more time, as every combination of hyperparameters is being tested.

---

## Setting up a Hyperparameter Grid

To find the best hyperparameters, we'll need to test various combinations. This is where the hyperparameter grid comes in. We define a set of values to test for each hyperparameter.

Here are the key hyperparameters we'll tune:

- **learning_rate**: This controls the contribution of each tree to the final prediction. A smaller learning rate means the model learns more slowly but can achieve better performance with proper tuning.
- **n_estimators**: This is the number of boosting stages (trees) to be used in the model. More boosting stages can improve performance but may also lead to overfitting.
- **max_depth**: This determines the maximum depth of the trees. Deeper trees can capture more complex patterns but may also overfit the training data.

Here's how to set up a hyperparameter grid:

```python
# Setting up the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 4]
}
```

In this grid, each combination of `learning_rate`, `n_estimators`, and `max_depth` will be tested. In this `param_grid` dictionary, the keys are hyperparameter names, and mapped lists contain their possible values.

---

## Implementing GridSearchCV

GridSearchCV automates the process of hyperparameter tuning by searching for the best combination of parameters in our grid.

Here's how to implement GridSearchCV:

```python
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate the GridSearchCV object
model = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3)

# Fit the model to the training data
model.fit(X_train, y_train)
```

In the code above, we first import the necessary libraries. We then instantiate the `GridSearchCV` object with a `GradientBoostingRegressor` and our predefined `param_grid`. The `cv=3` parameter specifies that 3-fold cross-validation should be used, meaning the data will be split into three subsets, and the model will be trained and validated three times, each time using a different subset for validation and the remaining subsets for training. This helps ensure the model's performance is robust and not dependent on a particular train-test split. Finally, we fit the `GridSearchCV` object to the training data, which involves training multiple models using different hyperparameter combinations and selecting the best one based on cross-validation results.

---

## Evaluating and Interpreting Results

Once GridSearchCV has found the best parameters, we need to evaluate and interpret the results.

```python
# Print the best parameters found
print("Best parameters found:", model.best_params_)
# Output:
# Best parameters found: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100}
```

Now, using the combination of hyperparameters that resulted in the best model performance, let's calculate the error using these parameters:

```python
# Predict with the best estimator
best_model = model.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print Mean Squared Error
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)
# Output:
# Mean Squared Error with best params: 22.27547097230719
```

This indicates the model's accuracy using the optimized hyperparameters, with a lower MSE indicating better accuracy.

---

## Visualizing Predictions

Visualizing predictions helps us understand how well our model is performing and identify any patterns or discrepancies between the actual and predicted values. By plotting the actual values against the predicted values, we can visually assess the model's accuracy and spot areas where the predictions may be off. This is crucial for interpreting the effectiveness of our hyperparameter tuning and understanding the model's behavior.

```python
# Plotting predictions vs actual values
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

Here, we visualize the comparison between actual values and predictions. The closer these points are together, the better the model's predictive performance.

---

## Lesson Summary

Great job! You've now learned how to use GridSearchCV for hyperparameter tuning to optimize a Gradient Boosting model. This process involves defining a hyperparameter grid, implementing GridSearchCV, and evaluating the results. Applying these techniques will significantly enhance your model's performance and ensure more accurate predictions.


## Adjusting Cross-Validation Folds in GridSearchCV

To tweak your `GridSearchCV` settings as requested:

1. Change the `cv` parameter from 2 to 4 to perform 4-fold cross-validation.
2. Modify the hyperparameter grid to include different options for `learning_rate` and `n_estimators`.

Here's the updated code with these adjustments:

```python
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Feature Engineering
tesla_df['SMA_5'] = tesla_df['Adj Close'].rolling(window=5).mean()
tesla_df['SMA_10'] = tesla_df['Adj Close'].rolling(window=10).mean()
tesla_df['EMA_5'] = tesla_df['Adj Close'].ewm(span=5, adjust=False).mean()
tesla_df['EMA_10'] = tesla_df['Adj Close'].ewm(span=10, adjust=False).mean()

# Drop NaN values created by moving averages
tesla_df.dropna(inplace=True)

# Select features and target
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_10', 'EMA_5', 'EMA_10']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values  # Predicting next day's close price
features = features[:-1]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# Setting up the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],  # Added 0.2 as an additional learning rate option
    'n_estimators': [100, 200, 300],   # Added 300 as an additional number of estimators option
    'max_depth': [3, 4]
}

# Instantiate the GridSearchCV object with 4-fold cross-validation
model = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=4)

# Fit the model to the training data
model.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:", model.best_params_)

# Predict with the best estimator
best_model = model.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)

# Plotting predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

### Changes Made:
- **Cross-Validation (`cv`):** Changed from `cv=2` to `cv=4` for 4-fold cross-validation.
- **Hyperparameter Grid (`param_grid`):** Added additional options for `learning_rate` (`0.2`) and `n_estimators` (`300`) to the grid for more comprehensive model tuning.

These adjustments should help in improving the model's validation accuracy by exploring a wider range of hyperparameter combinations and using a more robust cross-validation strategy.

## Hyperparameter Tuning Debugging with GridSearchCV

Hey there, Space Explorer! 🚀

I've identified the issue in your code. The problem is that you intended to use `GridSearchCV` for hyperparameter tuning, but instead, you directly instantiated and fitted the `GradientBoostingRegressor` without actually using `GridSearchCV`. Additionally, you tried to access `model.best_params_`, which only exists if you used `GridSearchCV`.

Let me fix the code for you:

```python
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Feature Engineering
tesla_df['SMA_5'] = tesla_df['Adj Close'].rolling(window=5).mean()
tesla_df['SMA_10'] = tesla_df['Adj Close'].rolling(window=10).mean()
tesla_df['EMA_5'] = tesla_df['Adj Close'].ewm(span=5, adjust=False).mean()
tesla_df['EMA_10'] = tesla_df['Adj Close'].ewm(span=10, adjust=False).mean()

# Drop NaN values created by moving averages
tesla_df.dropna(inplace=True)

# Select features and target
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_10', 'EMA_5', 'EMA_10']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values  # Predicting next day's close price

# Ensure alignment
features = features[:-1]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# Setting up the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 4]
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=4)

# Fit the model to the training data using GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best parameters found:", grid_search.best_params_)

# Predict with the best estimator found by GridSearchCV
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)

# Plotting predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

### Key Fixes:
- **Replaced `GradientBoostingRegressor` with `GridSearchCV`:** The model instantiation now uses `GridSearchCV` to perform hyperparameter tuning.
- **Accessing Best Parameters:** The correct method to find the best parameters is by using `grid_search.best_params_`.
- **Predicting with the Best Model:** The predictions are made using `grid_search.best_estimator_`, which is the model with the best hyperparameters found during the search.

This should now run smoothly and perform hyperparameter tuning as intended! 🚀

## Hyperparameter Tuning and Model Visualization

Hey there, Space Explorer! 🚀

I've identified the issue in your code. The problem is that you intended to use `GridSearchCV` for hyperparameter tuning, but instead, you directly instantiated and fitted the `GradientBoostingRegressor` without actually using `GridSearchCV`. Additionally, you tried to access `model.best_params_`, which only exists if you used `GridSearchCV`.

Let me fix the code for you:

```python
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Feature Engineering
tesla_df['SMA_5'] = tesla_df['Adj Close'].rolling(window=5).mean()
tesla_df['SMA_10'] = tesla_df['Adj Close'].rolling(window=10).mean()
tesla_df['EMA_5'] = tesla_df['Adj Close'].ewm(span=5, adjust=False).mean()
tesla_df['EMA_10'] = tesla_df['Adj Close'].ewm(span=10, adjust=False).mean()

# Drop NaN values created by moving averages
tesla_df.dropna(inplace=True)

# Select features and target
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_5', 'SMA_10', 'EMA_5', 'EMA_10']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values  # Predicting next day's close price

# Ensure alignment
features = features[:-1]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# Setting up the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.1],
    'n_estimators': [100, 200],
    'max_depth': [3, 4]
}

# Instantiate the GridSearchCV object
grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=4)

# Fit the model to the training data using GridSearchCV
grid_search.fit(X_train, y_train)

# Print the best parameters found by GridSearchCV
print("Best parameters found:", grid_search.best_params_)

# Predict with the best estimator found by GridSearchCV
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)

# Plotting predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

### Key Fixes:
- **Replaced `GradientBoostingRegressor` with `GridSearchCV`:** The model instantiation now uses `GridSearchCV` to perform hyperparameter tuning.
- **Accessing Best Parameters:** The correct method to find the best parameters is by using `grid_search.best_params_`.
- **Predicting with the Best Model:** The predictions are made using `grid_search.best_estimator_`, which is the model with the best hyperparameters found during the search.

This should now run smoothly and perform hyperparameter tuning as intended! 🚀

## GridSearchCV for Hyperparameter Tuning

The stars are aligning, Stellar Navigator! Let's complete the code so you can find the best hyperparameters and shine bright in your data journey. Here’s how you can fill in the blanks:

```python
import pandas as pd
from datasets import load_dataset
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Feature Engineering
tesla_df['Price_Range'] = tesla_df['High'] - tesla_df['Low']
tesla_df['Daily_Return'] = tesla_df['Adj Close'] / tesla_df['Open'] - 1
tesla_df['Volatility'] = tesla_df['Price_Range'].rolling(window=5).std()
tesla_df['Average_Volume'] = tesla_df['Volume'].rolling(window=5).mean()

# Drop NaN values created by rolling operations
tesla_df.dropna(inplace=True)

# Select features and target
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'Price_Range', 'Daily_Return', 'Volatility', 'Average_Volume']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values  # Predicting next day's close price
features = features[:-1]

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# Set up the hyperparameter grid
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5]
}

# Instantiate the GridSearchCV object
model = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3)

# Fit the model to the training data
model.fit(X_train, y_train)

# Print the best parameters found
print("Best parameters found:", model.best_params_)

# Predict with the best estimator
best_model = model.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print Mean Squared Error
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)

# Plot predictions vs actual values
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

### Key Points to Add:
1. **Hyperparameter Grid (`param_grid`):**
   - Expanded the range of `learning_rate`, `n_estimators`, and `max_depth` to explore a broader hyperparameter space.

2. **Model Fitting:**
   - Fit the `GridSearchCV` model using `model.fit(X_train, y_train)`.

3. **Best Parameters & Prediction:**
   - Extracted the best model and parameters using `model.best_params_` and `model.best_estimator_`.
   - Predicted with the best estimator and calculated the mean squared error.

4. **Visualization:**
   - Plotted the predictions against the actual values to visually compare model performance.

With these stellar adjustments, your hyperparameter tuning should be out of this world! 🌟

## Hyperparameter Tuning with GridSearchCV

Here’s your fully implemented code to optimize a Gradient Boosting model using the Tesla dataset. This code covers everything from loading the dataset, calculating technical indicators, setting up and training the model with `GridSearchCV`, and finally evaluating and visualizing the results.

```python
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from datasets import load_dataset

# Load dataset
tesla = load_dataset('codesignal/tsla-historic-prices')
tesla_df = pd.DataFrame(tesla['train'])

# Calculate 20-day Simple Moving Averages (SMA) on the dataset
tesla_df['SMA_20'] = tesla_df['Adj Close'].rolling(window=20).mean()

# Drop rows with NaN values after calculating the moving averages
tesla_df.dropna(inplace=True)

# Select relevant features and prepare the target variable
# The target will be - predicting next day's `Adj Close` price
features = tesla_df[['Open', 'High', 'Low', 'Close', 'Volume', 'SMA_20']].values
target = tesla_df['Adj Close'].shift(-1).dropna().values

# Align features with the target
features = features[:-1]

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.25, random_state=42)

# Set up the hyperparameter grid for learning_rate, n_estimators, and max_depth
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 150, 200],
    'max_depth': [3, 4, 5]
}

# Instantiate the GridSearchCV object with GradientBoostingRegressor
model = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3)

# Fit the GridSearchCV object to the training data
model.fit(X_train, y_train)

# Print best parameters found by GridSearchCV
print("Best parameters found:", model.best_params_)

# Use the best estimator to predict on the test set
best_model = model.best_estimator_
predictions = best_model.predict(X_test)

# Calculate and print the Mean Squared Error for the predictions
mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error with best params:", mse)

# Plot predictions against actual values using matplotlib
plt.figure(figsize=(10, 6))
plt.scatter(range(len(y_test)), y_test, label='Actual', alpha=0.7)
plt.scatter(range(len(y_test)), predictions, label='Predicted', alpha=0.7)
plt.title('Actual vs Predicted Values with Tuned Hyperparameters')
plt.xlabel('Sample Index')
plt.ylabel('Value')
plt.legend()
plt.show()
```

### Explanation of the Steps:

1. **Loading the Dataset**: The dataset is loaded using `load_dataset`, and converted into a DataFrame for easier manipulation.

2. **Calculating Technical Indicators**: A 20-day Simple Moving Average (SMA) is calculated on the adjusted closing price (`Adj Close`).

3. **Cleaning the Data**: Rows with NaN values, which result from the moving average calculation, are dropped to ensure no missing data during training.

4. **Preparing Features and Target**: Features include the stock's open, high, low, close, volume, and the newly calculated SMA. The target is set to the next day's adjusted closing price.

5. **Splitting Data**: The dataset is split into training and testing sets to evaluate the model's performance.

6. **Setting Up GridSearchCV**: A grid of hyperparameters is defined for `learning_rate`, `n_estimators`, and `max_depth`. `GridSearchCV` is used to find the optimal combination of these hyperparameters.

7. **Training the Model**: The model is trained on the training data with cross-validation.

8. **Evaluating the Model**: The best parameters are printed, predictions are made on the test set, and the Mean Squared Error (MSE) is calculated to measure the model's performance.

9. **Visualizing Results**: The actual vs. predicted values are plotted to visualize how well the model performs.

This complete implementation should help you fine-tune the Gradient Boosting model effectively. Happy tuning, Space Voyager! 🚀